In [ ]:
# 랭체인, 크로마db, 판다스 설치 진행
pip install pandas

In [ ]:
pip install sentence-transformers

In [ ]:
pip install langchain

In [ ]:
pip install -U langchain-community

In [ ]:
pip install chromadb

In [5]:
pwd

'/home/students/cs/202020933/rag'

In [ ]:
import pandas as pd

# CSV 파일 로드
data = pd.read_csv('./food_data.csv')
print(data.head())

  CKG_NM                                        CKG_MTRL_CN
0  어묵김말이  [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...
1  두부새우전  [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...
2     알밥  [재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...
3  현미호두죽           [재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간
4   북어갈비  [재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...


In [3]:
# 데이터에서 음식 이름과 재료만 뽑아서 200개만 저장
name = data["CKG_NM"].tolist()
ingredients = data["CKG_MTRL_CN"].tolist()

print(len(name))
print(len(ingredients))

name_list = name[0:200]
ingredients_list = ingredients[0:200]

print(len(name_list))
print(len(ingredients_list))
print(name_list[199])
print(ingredients_list[199])

224
224
200
200
삼겹살파인애플말이
[재료] 삼겹살 4줄| 통조림 파인애플 3쪽 [양념] 다시마물 1큰술| 간장 2큰술| 설탕 1큰술| 청주 1큰술| 생강즙 약간


In [ ]:
# HuggingFaceEmbeddings: Langchain에서 제공하는 클래스로, Hugging Face의 모델을 사용하여 텍스트를 벡터로 임베딩하는 역할
# 이 클래스는 Hugging Face에서 제공하는 다양한 모델을 쉽게 사용할 수 있게 해줌
from langchain.embeddings import HuggingFaceEmbeddings

# Chroma: Langchain에서 제공하는 벡터 데이터베이스. 이 클래스는 텍스트 임베딩을 저장하고, 나중에 임베딩을 기반으로 검색을 수행할 수 있게 해줌
from langchain.vectorstores import Chroma

# 모델 지정 모델은 허깅페이스의 intfloat/multilingual-e5-large-instruct 활용
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

# 크로마 db에 저장
# from_texts: Chroma 클래스의 from_texts 메소드를 사용하여 임베딩된 텍스트들을 Chroma DB에 저장하는 과정입니다. 이 메소드는 3개의 주요 인자를 받음

# ingredients_list: 텍스트 데이터 리스트입니다. ingredients_list는 임베딩을 적용할 원본 텍스트 데이터(예: 재료 목록 또는 텍스트의 다른 항목)를 포함하는 리스트, 이 텍스트들이 embedding_model을 통해 벡터로 변환
# embedding_model: 앞서 설정한 임베딩 모델을 사용하여 ingredients_list에 있는 각 텍스트를 고차원 벡터로 변환
# metadatas: 메타데이터를 지정하는 부분으로 각 텍스트에 대한 메타데이터를 제공할 수 있음. 여기서는 name_list에 있는 이름을 이용하여 각 텍스트에 대해 "Description": name 형태로 메타데이터를 생성하여, Chroma DB에 저장할 때 벡터와 함께 해당 텍스트에 대한 정보를 추가
db = Chroma.from_texts(ingredients_list, embedding_model, metadatas=[{"Description": name} for name in name_list])

하지만 이런 경우엔 임베딩 되는 내용에 음식 이름이 들어가지 않기 때문에 아래와 같이 질문을 진행하게 되면 음식 이름과 재료를 유사도 검사하여 문서를 가져오기 때문에 정확하지 않을 수 있음.
따라서 정확성을 높이기 위해서는 반드시 유사도 검색을 진행하기 위해 벡터화 하는 내용을 다음과 같이 수정하면 좋음

수정 내용 = "음식 이름"의 "재료"
EX) 어묵김말이의 재료 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T

In [6]:
#사용자의 질문
query = "어묵김말이의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 어묵김말이
ingredients: [재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T


In [7]:
#사용자의 질문
query = "두부새우전의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 두부새우전
ingredients: [재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개


In [8]:
#사용자의 질문
query = "현미호두죽의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 현미호두죽
ingredients: [재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간


In [37]:
#사용자의 질문
query = "삼겹살파인애플말이의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 삼겹살파인애플말이
ingredients: [재료] 삼겹살 4줄| 통조림 파인애플 3쪽 [양념] 다시마물 1큰술| 간장 2큰술| 설탕 1큰술| 청주 1큰술| 생강즙 약간


In [10]:
#사용자의 질문
query = "양파링튀김의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 양파링튀김
ingredients: [재료] 양파1개| 빵가루| 달걀 노른자1개| 밀가루1C| 물 약간| 파슬리 


In [11]:
#사용자의 질문
query = "베이컨김치말이밥의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 베이컨김치말이밥
ingredients: [재료] 밥 1공기| 김치 1/6포기| 베이컨 적당량 | 깻잎 5장| 양파 1/4개


In [12]:
#사용자의 질문
query = "뚝배기라면의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 뚝배기라면
ingredients: [재료] 좋아하는 종류의 라면 1봉지| 김치 1/4포기| 굴 5~6개| 파 1개| 청or홍고추 1개 [양념] 고춧가루 약간


In [13]:
#사용자의 질문
query = "꼬마김밥의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 꼬마김밥
ingredients: [재료] 밥 2공기| 김밥용어묵| 오이 1/2개| 캔햄이나김밥용햄| 김밥용단무지| 김 [양념] 참기름 1T| 맛소금 1T


In [14]:
#사용자의 질문
query = "뮈슬리초코칩쿠키의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 뮈슬리초코칩쿠키
ingredients: [재료] 박력분 250그램| 버터 135그램| 베이킹 파우더 4그램| 베이킹 소다 2그램| 계란 1개| 설탕 125그램| 초코칩 200그램| 유기농 초코 뮈슬리 100~150그램


In [15]:
#사용자의 질문
query = "대추스콘의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=3)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 대추케이크
ingredients: [재료] 대추 170g| 두유 250g| 포도씨유 80g| 베이킹소다 1/2ts| 베이킹파우더 1ts| 우리밀통밀 150g| 호두 80g| 물 200g [양념] 계피가루 1/4ts| 설탕 120g| 소금 1/4ts
Description: 찹쌀경단
ingredients: [재료] 찹쌀가루 300g| 물 11스푼| 고운팥앙금 적당량| 노랑카스테라가루 약간| 연두색카스테라가루 약간| 흑임자가루 약간| 계피가루 약간| 생밤 1개| 대추 4개
Description: 대추스콘
ingredients: [재료] 강력분 150그램| 박력분 150그램| 설탕 20그램| 소금 2그램| 베이킹 파우더 5그램| 버터 45그램| 건포도 6그램| 달갈 1개| 우유 100ml| 대추야자 씨뺀것 70그램| 계란물 약간


In [16]:
#사용자의 질문
query = "사천짜장떡볶이의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 사천짜장떡볶이
ingredients: [재료] 떡볶이떡 400~500그램 정도| 양파 1/2개| 양배추 큰것 1/8통| 대파 2줄기| 어묵 2장| 쫄면 200그램 정도| 물 또는 육수 1+1/2컵 [양념] 다*다(소고기맛 조미료) 1/2큰술| 춘장 2큰술| 고추장 1큰술| 설탕 2큰술| 다진마늘 1큰술


In [17]:
#사용자의 질문
query = "스팸오이식빵롤의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 스팸오이식빵롤
ingredients: [재료] 식빵 4장| 스팸 1/3캔| 오이 1개| 체다치즈(노란치즈) [양념] 머스터드| 피클| 마요네즈


In [29]:
#사용자의 질문
query = "새우볶음밥고로케의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 새우볶음밥고로케
ingredients: [재료] 새우 10마리| 밥 2공기| 김치 1/4포기| 당근 1/2개| 감자 1/2개| 양파 1/2개| 빵가루| 달걀| 밀가루


In [19]:
#사용자의 질문
query = "카레새우삼각김밥의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 카레새우삼각김밥
ingredients: [재료] 새우 7마리| 양파 1/4개| 김치 1/6포기| 밥지을쌀| 김밥용김| 계란| 밀가루| 빵가루 [양념] 백세 카레가루 2T


In [20]:
#사용자의 질문
query = "호두가래떡구이의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 호두가래떡구이
ingredients: [재료] 가래떡 1개| 호두 15알 정도 [양념] 깨 3T| 꿀 5T


In [21]:
#사용자의 질문
query = "오코노미야키의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 오코노미야키
ingredients: [재료] 양배추 1/2통| 숙주 1줌| 오징어 1마리| 새우 6마리| 베이컨 2줄| 계란 1개| 실파 약간 [오코노미야키반죽] 밀가루 1C| 물 1C| 혼다시 1T| 후추&소금 약간 [양념] 돈가스소스| 마요네즈


In [22]:
#사용자의 질문
query = "묵은지꽁치조림의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 묵은지꽁치조림
ingredients: [재료] 꽁치| 묵은지| 무 [양념] 고춧가루| 후춧가루| 다진마늘


In [32]:
#사용자의 질문
query = "낙지볶음의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 낙지볶음
ingredients: [낙지볶음 재료] 낙지 3마리| 대파 2대| 매운고추 2개| 배추 3잎| 당근 약간| 양파 1/2개| 새송이버섯 2개| 참기름 1작은술| 삶은계란 4개 [과메기 재료] 배추| 다시마| 김| 청양고추 [낙지볶음 양념] 고추가루 3큰술| 고추장 2큰술| 간장 2큰술| 설탕 2큰술| 물엿 1큰술| 다진 마늘 1큰술| 들기름 1큰술| 표고가루 1큰술| 청주 2큰술 [과메기 양념] 초고추장


In [24]:
#사용자의 질문
query = "흑미와플의 재료는?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 흑미와플
ingredients: [재료] 흑미가루 120g| 노른자 2개| 설탕 1큰술| 소금 1/3작은술| 바닐라오일 약간| 베이킹 파우더 1작은술| 우유 100ml| 버터 60g| 흰자 2개| 설탕 1큰술


In [25]:
#사용자의 질문
query = "닭볶음탕의 재료는 뭐야?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 닭볶음탕
ingredients: [재료] 닭 1마리| 마늘 6쪽| 양파 1개| 감자| 당근| 버섯| 파프리카| 파| 청량고추 [양념] 고추장 2큰술| 고추가루 3큰술| 간장 1.5 - 2큰술| 물엿 2큰술| 다진마늘 2큰술| 청양고추다진것| 요리술 1/2컵


In [26]:
#사용자의 질문
query = "카레의 재료는 뭐야?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 카레
ingredients: [재료] 돼지고기| 당근| 감자| 파프리카| 양파| 올리브유 [양념] 시즈닝솔트| 후춧가루| 파키스탄카레| 한국카레| 일본카레


In [27]:
#사용자의 질문
query = "초코칩머핀의 재료는 뭐야?"

#질문에 유사한 1개의 문서를 뽑아서 results에 저장
results = db.similarity_search(query, k=1)

#results의 데이터 출력 진행
for result in results:
    print(f"Description: {result.metadata['Description']}")
    print(f"ingredients: {result.page_content}")

Description: 초코칩머핀
ingredients: [재료] 루프 유기농 초코칩 머핀믹스 1개| 올리브유나 카놀라유 150ml| 계란 3개| 우유 50ml [양념] 코팅용 초콜렛 200g| 아몬드 다진것 적당히
